# Data Cleaning for LightSpeeders

Cleaning will be executed on data for a fictitous company, LightSpeeders, and used for Tableau visualization. This process will include importing the data, renaming items to be more intuitive, finding missing values and outliers then properly handling them, and finally exporting the data into a CSV file.

<blockquote>Comments marked with a verticle bar above/below the colored blocks will explain the purpose or results of the section.</blockquote>

Analysis and Tableau visualizations can be found here:

[https://public.tableau.com/app/profile/kevinvchin/viz/LightSpeeders/LightSpeeders](https://public.tableau.com/app/profile/kevinvchin/viz/LightSpeeders/LightSpeeders)

<blockquote>Load Python libraries for data cleaning.</blockquote>

In [1]:
# for data manipulation
import numpy as np
import pandas as pd

# for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# for date/time manipulation
import datetime

# for distance calculation on earth
from math import radians, sin, cos, sqrt, asin

# for location manipulation
import os

# set path to folder with CSV files
folder_path = 'raw_data/'

# for displaying all of the columns in dataframes, preventing Juptyer from redacting them
pd.set_option('display.max_columns', None)

### Import Data and Display Details

<blockquote>Get a list of CSV files in the `folder_path` location.</blockquote>

In [2]:
# get list of CSV files in the path
csv_files = [files for files in os.listdir(folder_path) if files.endswith('.csv')]

<blockquote>Load the 12 CSV files from the `folder_path` into a dataframe. Each CSV file represents a month of data.</blockquote>

In [3]:
# initialize empty dataframe to store the combined data from CSV files
combined_data = pd.DataFrame()

# iterate through `csv_files` and combine the data into 1 dataframe
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)
    combined_data = pd.concat([combined_data, data], ignore_index=True)

<blockquote>Display combined data.</blockquote>

In [4]:
# display details
combined_data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,6F1682AC40EB6F71,electric_bike,2023-06-05 13:34:12,2023-06-05 14:31:56,NaN,NaN,NaN,NaN,41.910000,-87.690000,41.91,-87.70,member
1,622A1686D64948EB,electric_bike,2023-06-05 01:30:22,2023-06-05 01:33:06,NaN,NaN,NaN,NaN,41.940000,-87.650000,41.94,-87.65,member
2,3C88859D926253B4,electric_bike,2023-06-20 18:15:49,2023-06-20 18:32:05,NaN,NaN,NaN,NaN,41.950000,-87.680000,41.92,-87.63,member
3,EAD8A5E0259DEC88,electric_bike,2023-06-19 14:56:00,2023-06-19 15:00:35,NaN,NaN,NaN,NaN,41.990000,-87.650000,41.98,-87.66,member
4,5A36F21930D6A55C,electric_bike,2023-06-19 15:03:34,2023-06-19 15:07:16,NaN,NaN,NaN,NaN,41.980000,-87.660000,41.99,-87.65,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5743273,9FC228393F5FFEBF,electric_bike,2024-05-21 16:02:09,2024-05-21 16:06:58,Clarendon Ave & Leland Ave,TA1307000119,NaN,NaN,41.967860,-87.650089,41.96,-87.65,member
5743274,B939C8F50D7BAB8F,electric_bike,2024-05-28 05:30:46,2024-05-28 05:35:51,Wabash Ave & Roosevelt Rd,TA1305000002,NaN,NaN,41.867148,-87.625999,41.87,-87.64,member
5743275,53138A05B0B81C07,electric_bike,2024-05-21 18:05:12,2024-05-21 18:14:05,DuSable Lake Shore Dr & Belmont Ave,TA1309000049,NaN,NaN,41.940771,-87.639185,41.92,-87.63,member
5743276,05D362D8910B1234,electric_bike,2024-05-17 14:47:31,2024-05-17 14:58:45,Green St & Washington Blvd,13053,NaN,NaN,41.883253,-87.648854,41.90,-87.64,member


In [5]:
# show info for dataframe
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5743278 entries, 0 to 5743277
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 569.6+ MB


<blockquote>Details show that there are 5,743,278 rows of 13 columns.
<br><br>
The start latitude/longitude and end latitude/longitude are decimals, which are correct because they are coordinates. The other columns are object (string) type so those are also correct.
<br><br>
`started_at` and `ended_at` may need to be converted to datatime datatype later.</blockquote>

### Handle Missing Values and Duplicates

<blockquote>Find:

- number of rows with missing data
- number of missing data in each column
- rows that have missing data then print them
</blockquote>

In [6]:
# check for missing values
print('Number of rows with missing data: ', combined_data.isna().any(axis=1).sum(), sep='')
print()
print('Number of missing data in each column:')
print(combined_data.isna().sum(axis=0))
print()
print('Row(s) with missing data:')
print(combined_data[combined_data.isna().any(axis=1)])

Number of rows with missing data: 1428498

Number of missing data in each column:
ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    905237
start_station_id      905237
end_station_name      956579
end_station_id        956579
start_lat                  0
start_lng                  0
end_lat                 7684
end_lng                 7684
member_casual              0
dtype: int64

Row(s) with missing data:
                  ride_id  rideable_type           started_at  \
0        6F1682AC40EB6F71  electric_bike  2023-06-05 13:34:12   
1        622A1686D64948EB  electric_bike  2023-06-05 01:30:22   
2        3C88859D926253B4  electric_bike  2023-06-20 18:15:49   
3        EAD8A5E0259DEC88  electric_bike  2023-06-19 14:56:00   
4        5A36F21930D6A55C  electric_bike  2023-06-19 15:03:34   
...                   ...            ...                  ...   
5743273  9FC228393F5FFEBF  electric_bike  2024

<blockquote>There are 1,428,498 rows with missing data.</blockquote>

<blockquote>It's possible that a rental didn't start or end at a particular station, resulting in missing values for `start_station_name`, `start_station_id`, `end_station_name`, and `end_station_id`. Bikes rented or left on the sidewalk would have GPS coordinates, but not a station name or ID. Knowing the station of origin and destination will help market to potential new members so the rows with empty station name or ID will be removed</blockquote>

In [7]:
# remove rows with missing values then reindex
combined_data = combined_data.dropna(axis=0).reset_index(drop=True)

<blockquote>Check for duplicate rows.</blockquote>

In [8]:
# check for duplicates
print('Number duplicated data: ', combined_data.duplicated().sum(), sep='')

Number duplicated data: 0


<blockquote>Copy current progress into new dataframe</blockquote>

In [9]:
# make copy
df0 = combined_data.copy(deep=True)
df0.shape

(4314780, 13)

### Add New Columns Through Data Transformation

<blockquote>Calculate ride duration (in seconds) and add result as a new column named `ride_duration_s`.</blockquote

In [10]:
# convert `started_at` and `ended_at` to datetime datatype
date_columns = ['started_at', 'ended_at']
df0[date_columns] = df0[date_columns].apply(pd.to_datetime)

# calculate ride duration (seconds) and store result in a new column
df0['ride_duration_s'] = (df0['ended_at'] - df0['started_at']).dt.total_seconds()
df0.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration_s
0,055E6783FCD761FD,electric_bike,2023-06-06 20:58:13,2023-06-06 21:04:00,California Ave & Milwaukee Ave,13084,California Ave & Division St,13256,41.922618,-87.697106,41.903029,-87.697474,member,347.0
1,65BC2A1FC12CFFA4,classic_bike,2023-06-28 18:21:26,2023-06-28 18:56:06,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,2080.0
2,4D149B57532FE9C9,classic_bike,2023-06-30 09:53:16,2023-06-30 09:53:38,Western Ave & Roscoe St,15634,Western Ave & Roscoe St,15634,41.943034,-87.687288,41.943034,-87.687288,member,22.0
3,BAE48AFCA5C36414,classic_bike,2023-06-21 13:23:57,2023-06-21 13:24:16,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,19.0
4,E3491C5273700783,classic_bike,2023-06-21 13:24:46,2023-06-21 13:40:47,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,961.0


<blockquote>Check if `ride_duration_s` makes sense, logically. There shouldn't be any negative seconds.  However, what if the duration is over 24 hours (86,400 seconds)? It wouldn't be possible time-wise but it would be possible to rent the bike but not return it until days, or even weeks later.<blockquote>

In [11]:
# display statistical data for `ride_duration_s`
df0['ride_duration_s'].describe()

count    4.314780e+06
mean     9.806874e+02
std      2.204320e+03
min     -3.274000e+03
25%      3.440000e+02
50%      6.000000e+02
75%      1.072000e+03
max      6.691360e+05
Name: ride_duration_s, dtype: float64

<blockquote>Remove the rows with negative seconds and keep everything else.</blockquote>

In [12]:
# Keep `ride_duration_s` if greater than 0 seconds
df0_time = df0[df0['ride_duration_s'] > 0]
df1 = df0_time.copy(deep=True)
df1.shape

(4314114, 14)

<blockquote>Calculate ride distance in miles using Haversine formula and the coordinates `start_lat`, `start_lng`, `end_lat`, and `end_lng`. Since the earth is not flat, we have to use the formula to calculate distance on a round object.</blockquote>

In [13]:
# Haversine formula
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in miles between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 # Radius of earth in miles. Use 6371 for kilometers. Determines return value units.
    return c * r

In [14]:
# copy to a new dataframe
df2 = df1.copy(deep=True)

# Calculate distance and store the result in a new column named `distance_traveled_mi`
df2['distance_traveled_mi'] = df2.apply(lambda row: haversine(row['start_lng'], row['start_lat'], row['end_lng'], row['end_lat']), axis=1)
df2.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration_s,distance_traveled_mi
0,055E6783FCD761FD,electric_bike,2023-06-06 20:58:13,2023-06-06 21:04:00,California Ave & Milwaukee Ave,13084,California Ave & Division St,13256,41.922618,-87.697106,41.903029,-87.697474,member,347.0,1.352636
1,65BC2A1FC12CFFA4,classic_bike,2023-06-28 18:21:26,2023-06-28 18:56:06,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,2080.0,0.000000
2,4D149B57532FE9C9,classic_bike,2023-06-30 09:53:16,2023-06-30 09:53:38,Western Ave & Roscoe St,15634,Western Ave & Roscoe St,15634,41.943034,-87.687288,41.943034,-87.687288,member,22.0,0.000000
3,BAE48AFCA5C36414,classic_bike,2023-06-21 13:23:57,2023-06-21 13:24:16,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,19.0,0.000000
4,E3491C5273700783,classic_bike,2023-06-21 13:24:46,2023-06-21 13:40:47,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,961.0,0.000000


<blockquote>Check if `distance_traveled_mi` makes sense, logically. There shouldn't be any negative distances. It would also be impossible to ride thousands of miles. We'll check this later when calculating miles per hour.<blockquote>

In [15]:
# display statistical data for `ride_duration_s`
df2['distance_traveled_mi'].describe()

count    4.314114e+06
mean     1.290633e+00
std      5.210301e+00
min      0.000000e+00
25%      5.388169e-01
50%      9.503616e-01
75%      1.672481e+00
max      6.094779e+03
Name: distance_traveled_mi, dtype: float64

<blockquote>As expected, there is a maximum of 6,095 miles. We can determine if this is legitimate by calculating miles per hour.
<br><br>
Something of note is that there is a minimum value of 0 miles. This is possible if it was a round trip, meaning the user started and ended at the same location.</blockquote>

<blockquote>Calculate speed and add as a new column named `speed_mi`.</blockquote>

In [16]:
# copy to a new dataframe
df3 = df2.copy(deep=True)

# calculate speed
df3['speed_mih'] = (df3['distance_traveled_mi'] / (df3['ride_duration_s'] / 3600))

In [17]:
# display statistical data for 'speed_mih'
df3['speed_mih'].describe()

count    4.314114e+06
mean     6.538752e+00
std      2.957929e+02
min      0.000000e+00
25%      4.665559e+00
50%      6.534705e+00
75%      8.308162e+00
max      6.094779e+05
Name: speed_mih, dtype: float64

<blockquote>There is a maximum value of 60,9478 miles per hour! The average speed for professional cyclists on flat terrain is 25-28 miles per hour, while amatuers travel about 17-18 mph.</blockquote>

<blockquote>We will cap the speed at 28 mph and eliminate everything above it.</blockquote>

In [18]:
# Keep `speed_mih` if less than or equal to 28
df3 = df3[df3['speed_mih'] <= 28]
df3.shape

(4313045, 16)

<blockquote>Let's check the statistical breakdown of each column of the dataframe:

- count (number of times occured)
- unique (how many distinct values)
- top (most frequent occurance)
- freq (number of times more frequent has occured)
- mean (average)
- min (minimum value)
- 25% (value at the 25% interquartile)
- 50% (value at 50% interquartile, also known as median value)
- 75% (value at 75% interquartile)
- max (maximum value)
- std (standard deviation)</blockquote>

In [19]:
# describe dataframe
df3.describe(include='all')

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration_s,distance_traveled_mi,speed_mih
count,4313045,4313045,4313045,4313045,4313045,4313045,4313045,4313045,4.313045e+06,4.313045e+06,4.313045e+06,4.313045e+06,4313045,4.313045e+06,4.313045e+06,4.313045e+06
unique,4313045,3,NaN,NaN,1602,1572,1623,1591,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN
top,055E6783FCD761FD,classic_bike,NaN,NaN,Streeter Dr & Grand Ave,13022,Streeter Dr & Grand Ave,13022,NaN,NaN,NaN,NaN,member,NaN,NaN,NaN
freq,1,2822839,NaN,NaN,60525,60525,62063,62063,NaN,NaN,NaN,NaN,2798512,NaN,NaN,NaN
mean,NaN,NaN,2023-10-31 14:54:59.698592768,2023-10-31 15:11:20.771032832,NaN,NaN,NaN,NaN,4.190055e+01,-8.764442e+01,4.190100e+01,-8.764470e+01,NaN,9.810724e+02,1.286294e+00,6.354441e+00
min,NaN,NaN,2023-06-01 00:00:44,2023-06-01 00:02:56,NaN,NaN,NaN,NaN,4.164850e+01,-8.784411e+01,4.164850e+01,-8.784406e+01,NaN,1.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,NaN,2023-07-30 14:18:53,2023-07-30 14:43:39,NaN,NaN,NaN,NaN,4.188048e+01,-8.765715e+01,4.188096e+01,-8.765814e+01,NaN,3.450000e+02,5.389830e-01,4.664913e+00
50%,NaN,NaN,2023-09-28 18:26:52,2023-09-28 18:40:08,NaN,NaN,NaN,NaN,4.189577e+01,-8.764130e+01,4.189637e+01,-8.764170e+01,NaN,6.000000e+02,9.504741e-01,6.533962e+00
75%,NaN,NaN,2024-02-13 16:17:27,2024-02-13 16:29:49,NaN,NaN,NaN,NaN,4.192628e+01,-8.762783e+01,4.192871e+01,-8.762784e+01,NaN,1.072000e+03,1.672385e+00,8.306492e+00
max,NaN,NaN,2024-05-31 23:59:47,2024-06-01 17:56:00,NaN,NaN,NaN,NaN,4.206487e+01,-8.752823e+01,4.206485e+01,-8.752823e+01,NaN,6.691360e+05,2.082557e+01,2.794460e+01


<blockquote>
Statistical analysis cannot be done on Object datatypes. Recall that `ride_id`, `rideable_type`, `started_at`, `ended_at`, `start_station_name`, `start_station_id`, `end_station_name`, `end_station_id`, and `member_casual` are Object datatypes so there will be no information for mean, std, min, 25%, 50%, 75%, and max.
<br><br>
All values for number (float) datatype seem logically correct.
</blockquote>

<blockquote>Remove `ride_id` and `speed_mih` because we don't need them anymore. `speed_mih` was used to check for outliers.</blockquote>

In [20]:
# drop columns 'ride_id' and 'speed_mih'
columns_to_drop = ['ride_id', 'speed_mih']
df3_dropped = df3.drop(columns=columns_to_drop)
df3_dropped.shape

(4313045, 14)

<blockquote>Remove possible leading and trailing whitespaces from Object (string) datatypes.</blockquote>

In [21]:
# remove leading and trailing spaces
columns_to_strip = ['rideable_type', 'start_station_name', 'start_station_id',	'end_station_name', 'end_station_id', 'member_casual']
df3_dropped[columns_to_strip] = df3_dropped[columns_to_strip].astype(str).apply(lambda col: col.str.strip())

<blockquote>If `start_station_name` equals `end_station_name` then it was a round trip, otherwise it was a one-way trip. A new column named `route_type` will be added to contain results.</blockquote>

In [22]:
# function for `route_type`
def route_type (row):
  if row['start_station_name'] == row['end_station_name']:
    return "Round"
  else:
    return "One-way"

In [42]:
# copy to a new dataframe
df4 = df3_dropped.copy(deep=True)

# add `route_type` and column
df4['route_type'] = df4.apply(route_type, axis=1)
df4.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration_s,distance_traveled_mi,route_type
0,electric_bike,2023-06-06 20:58:13,2023-06-06 21:04:00,California Ave & Milwaukee Ave,13084,California Ave & Division St,13256,41.922618,-87.697106,41.903029,-87.697474,member,347.0,1.352636,One-way
1,classic_bike,2023-06-28 18:21:26,2023-06-28 18:56:06,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,2080.0,0.000000,Round
2,classic_bike,2023-06-30 09:53:16,2023-06-30 09:53:38,Western Ave & Roscoe St,15634,Western Ave & Roscoe St,15634,41.943034,-87.687288,41.943034,-87.687288,member,22.0,0.000000,Round
3,classic_bike,2023-06-21 13:23:57,2023-06-21 13:24:16,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,19.0,0.000000,Round
4,classic_bike,2023-06-21 13:24:46,2023-06-21 13:40:47,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,961.0,0.000000,Round


<blockquote>Calculate and add ride duration in minutes since that is more intuitive. New column name is `ride_duration_min`</blockquote>

In [43]:
# copy to a new dataframe
df5 = df4.copy(deep=True)

# add `ride_duration_min`
df5['ride_duration_min'] = df5['ride_duration_s'] / 60
df5.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration_s,distance_traveled_mi,route_type,ride_duration_min
0,electric_bike,2023-06-06 20:58:13,2023-06-06 21:04:00,California Ave & Milwaukee Ave,13084,California Ave & Division St,13256,41.922618,-87.697106,41.903029,-87.697474,member,347.0,1.352636,One-way,5.783333
1,classic_bike,2023-06-28 18:21:26,2023-06-28 18:56:06,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,2080.0,0.000000,Round,34.666667
2,classic_bike,2023-06-30 09:53:16,2023-06-30 09:53:38,Western Ave & Roscoe St,15634,Western Ave & Roscoe St,15634,41.943034,-87.687288,41.943034,-87.687288,member,22.0,0.000000,Round,0.366667
3,classic_bike,2023-06-21 13:23:57,2023-06-21 13:24:16,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,19.0,0.000000,Round,0.316667
4,classic_bike,2023-06-21 13:24:46,2023-06-21 13:40:47,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,961.0,0.000000,Round,16.016667


<blockquote>Remove `rider_duration_s` to speed up load time in Tableau.</blockquote>

In [44]:
df5_dropped = df5.drop(columns=['ride_duration_s'])

<blockquote>Rename and rearrange columns to be more intuitive.</blockquote>

In [45]:
# copy to a new dataframe
df_format = df5_dropped.copy(deep=True)

# rename columns and rearrange columns
df_format.rename(columns={'rideable_type': 'ride_type',
                          'started_at': 'start_time',
                          'ended_at': 'end_time',
                          'start_lng': 'start_lon',
                          'end_lng': 'end_lon',
                          'member_casual': 'user_type'
                           }, inplace=True)

rearranged_cols = ['user_type',
                   'ride_type',
                   'start_time',
                   'end_time',
                   'route_type',
                   'start_station_name',
                   'end_station_name',
                   'start_lat',
                   'start_lon',
                   'end_lat',
                   'end_lon',
                   'ride_duration_min',
                   'distance_traveled_mi'
                  ]

df_format = df_format[rearranged_cols].reset_index(drop=True)

<blockquote>Rename column values to make project more immersive.

`user_type`
- member -> Imperial
- casual -> Rebel

`ride_type`
- classic_bike -> Hover Bike
- electric_bike -> Speeder Bike
- docked_bike -> Landspeeder
</blockquote>

In [46]:
# copy to a new dataframe
df_rename = df_format.copy(deep=True)

# encode the `user_type` column values
df_rename['user_type'] = df_rename['user_type'].map({
                                         'member': 'Imperial',
                                         'casual': 'Rebel'
                                        })

In [48]:
# encode the `rider_type` column values
df_rename['ride_type'] = df_rename['ride_type'].map({
                                                    'classic_bike': 'Hover Bike',
                                                    'electric_bike': 'Speeder Bike',
                                                    'docked_bike': 'Landspeeder'
                                                    })

In [49]:
# copy to a new dataframe
df_clean = df_rename.copy(deep=True)

### Final Check

In [50]:
# show info for dataframe
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4313045 entries, 0 to 4313044
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   user_type             object        
 1   ride_type             object        
 2   start_time            datetime64[ns]
 3   end_time              datetime64[ns]
 4   route_type            object        
 5   start_station_name    object        
 6   end_station_name      object        
 7   start_lat             float64       
 8   start_lon             float64       
 9   end_lat               float64       
 10  end_lon               float64       
 11  ride_duration_min     float64       
 12  distance_traveled_mi  float64       
dtypes: datetime64[ns](2), float64(6), object(5)
memory usage: 427.8+ MB


In [51]:
# describe dataframe
df_clean.describe(include='all')

,user_type,ride_type,start_time,end_time,route_type,start_station_name,end_station_name,start_lat,start_lon,end_lat,end_lon,ride_duration_min,distance_traveled_mi
count,4313045,4313045,4313045,4313045,4313045,4313045,4313045,4.313045e+06,4.313045e+06,4.313045e+06,4.313045e+06,4.313045e+06,4.313045e+06
unique,2,3,NaN,NaN,2,1602,1623,NaN,NaN,NaN,NaN,NaN,NaN
top,Imperial,Hover Bike,NaN,NaN,One-way,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave,NaN,NaN,NaN,NaN,NaN,NaN
freq,2798512,2822839,NaN,NaN,4046041,60525,62063,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2023-10-31 14:54:59.698592768,2023-10-31 15:11:20.771032832,NaN,NaN,NaN,4.190055e+01,-8.764442e+01,4.190100e+01,-8.764470e+01,1.635121e+01,1.286294e+00
min,NaN,NaN,2023-06-01 00:00:44,2023-06-01 00:02:56,NaN,NaN,NaN,4.164850e+01,-8.784411e+01,4.164850e+01,-8.784406e+01,1.666667e-02,0.000000e+00
25%,NaN,NaN,2023-07-30 14:18:53,2023-07-30 14:43:39,NaN,NaN,NaN,4.188048e+01,-8.765715e+01,4.188096e+01,-8.765814e+01,5.750000e+00,5.389830e-01
50%,NaN,NaN,2023-09-28 18:26:52,2023-09-28 18:40:08,NaN,NaN,NaN,4.189577e+01,-8.764130e+01,4.189637e+01,-8.764170e+01,1.000000e+01,9.504741e-01
75%,NaN,NaN,2024-02-13 16:17:27,2024-02-13 16:29:49,NaN,NaN,NaN,4.192628e+01,-8.762783e+01,4.192871e+01,-8.762784e+01,1.786667e+01,1.672385e+00
max,NaN,NaN,2024-05-31 23:59:47,2024-06-01 17:56:00,NaN,NaN,NaN,4.206487e+01,-8.752823e+01,4.206485e+01,-8.752823e+01,1.115227e+04,2.082557e+01


<blockquote>Data looks good so we're going to export the dataframe to a .csv file. We'll also need to check the data types and correct accordingly once data is imported into Tableau.</blockquote>

### Export File

In [53]:
df_clean.to_csv('LightSpeeders.csv', index=False)

<blockquote>Done! 
<br>
File name is `LightSpeeders.csv` and the size is 849,103 KB.

See you in Tableau!</blockquote>

Analysis and Tableau visualizations can be found here:

[https://public.tableau.com/app/profile/kevinvchin/viz/LightSpeeders/LightSpeeders](https://public.tableau.com/app/profile/kevinvchin/viz/LightSpeeders/LightSpeeders)